<a href="https://colab.research.google.com/github/nik251/CV/blob/main/SHMQHM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import IPython

import webbrowser
import os

f = open('GFG.html', 'w')

html_template = """

<!DOCTYPE html>

<html>

<head>
	<meta charset="utf-8">
	<title>Quantum Harmonic Oscillator</title>
	<style>
	.custombutton {
		display: inline-block;
		width:92px;
		height:24px;
		line-height:24px;
		font-size:15px;
		font-family:Arial, sans-serif;
		text-align:center;
		color:black;
		background:-webkit-linear-gradient(white,#eeeeee,#eeeeee,#e0e0e0);
		background:linear-gradient(white,#eeeeee,#eeeeee,#e0e0e0);
		text-decoration:none;
		border:1px solid rgb(255, 255, 255);
		border-radius: 5px;
		-webkit-user-select: none;
		-moz-user-select: -moz-none;
		-ms-user-select: none;
		user-select: none;
		cursor: pointer;
		-webkit-tap-highlight-color: rgba(0,0,0,0);
	}
	.custombutton:active {
		background:-webkit-linear-gradient(#909090,#808080,#808080,#707070);
		background:linear-gradient(#909090,#808080,#808080,#707070);
	}
	input[type=range] {-webkit-user-select: none; user-select: none;}
	sup {position:relative; vertical-align:bottom; bottom:0.4em; font-size:0.75em;}
	</style>
</head>

<body>

<div style="width:80%; margin-left:auto; margin-right:auto;">

	<h1 style="font-size:18pt; text-align:center; margin-bottom:10px;">Quantum Harmonic Oscillator</h1>

	<canvas id="theCanvas" width="600" height="300">Error!</canvas>

	<div style="text-align:center; -webkit-text-size-adjust: 100%;">
		<a href="javascript:void(0)" class="custombutton" id="pauseButton" value="Pause"
				onclick="startStop();" ontouchstart="">Pause</a>
		&nbsp;Speed:
		<input type="range" id="speedSlider" min="0" max="0.2" step="0.001" value="0.05">
		&nbsp
		<input type="radio" name="plotType" id="realImag" onclick="paintCanvas();">Real/imag&nbsp;
		<input type="radio" name="plotType" checked onclick="paintCanvas();">Density/phase
	</div>
	<div style="text-align:center; -webkit-text-size-adjust: 100%; margin-top:4px;">
		<a href="javascript:void(0)" class="custombutton"
				onclick="zero();" ontouchstart="">Zero</a>
		<a href="javascript:void(0)" class="custombutton"
				onclick="normalizePsi();" ontouchstart="">Normalize</a>
		&nbsp;&nbsp;
		<a href="javascript:void(0)" class="custombutton"
				onclick="coherent();" ontouchstart="">Coherent(&alpha;)</a>
		&alpha; = <span id="alphaReadout">1.0</span>
		<input type="range" id="alphaSlider" min="0" max="4" step="0.1" value="1"
				onchange="adjustAlpha();" oninput="adjustAlpha();">
	</div>


        <p>circles display phasor diagrams for the complex amplitudes of  basis functions, with the ground state on the left [radius of magnitude 1]</p>

        <p>wavefunction construction: add the eight basis functions after they have been multiplied by their corresponding complex amplitude</p>

        <p>over time, each basis amplitude rotates in the complex plane at a frequency proportional to its corresponding energy</p>

        <p>real and imaginary parts are represented by orange and blue lines, respectively</p>

        <p>Probability density and phase are shown, with phase indicated by colors:</p>

            <p>->red - real and positive </p>
            <p>->light green - imaginary and positive</p>
            <p>->cyan -real and negative</p>
            <p>->purple - imaginary and negative</p>
            <p>click and drag lines on clock faces to adjust the corresponding amplitude.</p>

        <p>for an specific basis function: zero, then drag the line on the clock face.</p>

        <p>an approximation of a coherent state can also be created, mimicking the oscillation of a classical particle.</p>

        <p>α = the amplitude of this oscillation [and &alpha;<sup>2</sup> is the average number of energy units above ground state]</p>

        <p>for large values of &alpha;, the true coherent state is not accurately represented using only the lowest eight basis states</p>

</div>

<script>

	var theCanvas = document.getElementById("theCanvas");
	var theContext = theCanvas.getContext("2d");
	var pauseButton = document.getElementById("pauseButton");
	var speedSlider = document.getElementById("speedSlider");
	var realImag = document.getElementById("realImag");
	var alphaSlider = document.getElementById("alphaSlider");
	var alphaReadout = document.getElementById("alphaReadout");

	var iMax = Number(theCanvas.width);	// max index in function arrays (so array size is iMax+1)
	var pxPerX = 60;			// number of pixels per conventional x unit
	var clockSpaceFraction = 0.25;	// fraction of vertical space taken up by clocks
	var clockRadiusFraction = 0.45;	// as fraction of width or height of clock space
	var clockSpaceHeight = theCanvas.height * clockSpaceFraction;
	var clockPixelRadius = clockSpaceHeight * clockRadiusFraction;
	var psi = {re:(new Array(iMax+1)), im:(new Array(iMax+1))}
	var nMax = 7;				// maximum energy quantum number
	var eigenPsi = new Array(nMax+1);
	var amplitude = new Array(nMax+1);		// amplitudes of the eigenfunctions in psi
	var phase = new Array(nMax+1);			// phases of the eigenfunctions in psi
	var nColors = 360;
	var phaseColor = new Array(nColors+1);
	var running = true;
	var mouseIsDown = false;
	var mouseClock;

	// Add mouse/touch handlers; down/start must be inside the canvas but drag can go outside it:
	theCanvas.addEventListener('mousedown', mouseDown, false);
	document.body.addEventListener('mousemove', mouseMove, false);
	document.body.addEventListener('mouseup', mouseUp, false);	// button release could occur outside canvas
	theCanvas.addEventListener('touchstart', touchStart, false);
	document.body.addEventListener('touchmove', touchMove, false);
	document.body.addEventListener('touchend', mouseUp, false);

	init();
	nextFrame();

	function init() {
		// Initialize eigenfunctions (simple harmonic oscillator):
		for (var n=0; n<=nMax; n++) {
			eigenPsi[n] = new Array(iMax+1);
		}
		for (var i=0; i<=iMax; i++) {
			var x = (i - iMax/2) / pxPerX;
			eigenPsi[0][i] = Math.exp(-x*x/2);
			eigenPsi[1][i] = Math.sqrt(2) * x * eigenPsi[0][i];
			eigenPsi[2][i] = (1/Math.sqrt(2)) * (2*x*x - 1) * eigenPsi[0][i];
			eigenPsi[3][i] = (1/Math.sqrt(3)) * (2*x*x*x - 3*x) * eigenPsi[0][i];
			eigenPsi[4][i] = (1/Math.sqrt(24)) * (4*x*x*x*x - 12*x*x + 3) * eigenPsi[0][i];
			eigenPsi[5][i] = (1/Math.sqrt(60)) * (4*x*x*x*x*x - 20*x*x*x + 15*x) * eigenPsi[0][i];
			eigenPsi[6][i] = (1/Math.sqrt(720)) * (8*x*x*x*x*x*x - 60*x*x*x*x + 90*x*x - 15) * eigenPsi[0][i];
			eigenPsi[7][i] = (1/Math.sqrt(36*70)) * (8*x*x*x*x*x*x*x - 84*x*x*x*x*x + 210*x*x*x - 105*x) * eigenPsi[0][i];
		}
		// Initialize amplitudes and phases:
		for (var n=0; n<=nMax; n++) {
			amplitude[n] = 0;
			phase[n] = 0;
		}
		amplitude[0] = 1/Math.sqrt(2);
		amplitude[1] = 1/Math.sqrt(2);
		// Initialize array of colors to represent phases:
		for (var c=0; c<=nColors; c++) {
			phaseColor[c] = colorString(c/nColors);
		}
	}

	function nextFrame() {
		for (var n=0; n<=nMax; n++) {
			phase[n] -= (n+0.5) * Number(speedSlider.value);
			if (phase[n] < 0) phase[n] += 2*Math.PI;
		}
		buildPsi();
		paintCanvas();
		if (running) window.setTimeout(nextFrame, 1000/30);
	}

	function buildPsi() {
		for (var i=0; i<=iMax; i++) {
			psi.re[i] = 0;
			psi.im[i] = 0;
		}
		for (var n=0; n<=nMax; n++) {
			var realPart = amplitude[n] * Math.cos(phase[n]);
			var imagPart = amplitude[n] * Math.sin(phase[n]);
			for (var i=0; i<=iMax; i++) {
				psi.re[i] += realPart * eigenPsi[n][i];
				psi.im[i] += imagPart * eigenPsi[n][i];
			}
		}
	}

	function setMouseClock(relX, relY) {	// parameters are x,y in pixels, relative to clock center
		mouseIsDown = true;
		var pixelDistance = Math.sqrt(relX*relX + relY*relY);
		amplitude[mouseClock] = Math.min(pixelDistance / clockPixelRadius, 1);
		phase[mouseClock] = Math.atan2(relY, relX);
		if (phase[mouseClock] < 0) phase[mouseClock] += 2*Math.PI;
		buildPsi();
		paintCanvas();
	}

	function mouseOrTouchStart(pageX, pageY, e) {
		if (pageY-theCanvas.offsetTop > theCanvas.height - clockSpaceHeight) {
			mouseClock = Math.floor((pageX - theCanvas.offsetLeft) / clockSpaceHeight);
			var clockCenterX = clockSpaceHeight * (mouseClock + 0.5);	// relative to left of canvas
			var clockCenterY = theCanvas.height - clockSpaceHeight*0.5;	// relative to top of canvas
			var relX = pageX - theCanvas.offsetLeft - clockCenterX;
			var relY = clockCenterY - (pageY - theCanvas.offsetTop) ;	// measured up from clock center
			if (relX*relX + relY*relY <= clockPixelRadius*clockPixelRadius) {
				setMouseClock(relX, relY);
				e.preventDefault();
			}
		}
	}

	function mouseOrTouchMove(pageX, pageY, e) {
		if (mouseIsDown) {
			var clockCenterX = clockSpaceHeight * (mouseClock + 0.5);	// relative to left of canvas
			var clockCenterY = theCanvas.height - clockSpaceHeight*0.5;	// relative to top of canvas
			var relX = pageX - theCanvas.offsetLeft - clockCenterX;
			var relY = clockCenterY - (pageY - theCanvas.offsetTop) ;	// measured up from clock center
			setMouseClock(relX, relY);
			e.preventDefault();
		}
	}

	function mouseDown(e) {
		mouseOrTouchStart(e.pageX, e.pageY, e);
	}

	function touchStart(e) {
		mouseOrTouchStart(e.targetTouches[0].pageX, e.targetTouches[0].pageY, e);
	}

	function mouseMove(e) {
		mouseOrTouchMove(e.pageX, e.pageY, e);
	}

	function touchMove(e) {
		mouseOrTouchMove(e.targetTouches[0].pageX, e.targetTouches[0].pageY, e);
	}

	function mouseUp(e) {
		mouseIsDown = false;
		paintCanvas();
	}

	function paintCanvas() {
		theContext.fillStyle = "black";
		theContext.fillRect(0, 0, theCanvas.width, theCanvas.height);

		var baselineY, pxPerY;

		if (realImag.checked) {
			baselineY = theCanvas.height * (1 - clockSpaceFraction) / 2;
			pxPerY = baselineY * 0.9;

			// Draw the horizontal axis:
			theContext.strokeStyle = "gray";
			theContext.lineWidth = 1;
			theContext.beginPath();
			theContext.moveTo(0, baselineY);
			theContext.lineTo(theCanvas.width, baselineY);
			theContext.stroke();

			theContext.lineWidth = 2;

			// Plot the real part of psi:
			theContext.beginPath();
			theContext.moveTo(0, baselineY - psi.re[0]*pxPerY);
			for (var i=1; i<=iMax; i++) {
				theContext.lineTo(i, baselineY - psi.re[i]*pxPerY);
			}
			theContext.strokeStyle = "#ffc000";
			theContext.stroke();

			// Plot the imaginary part of psi:
			theContext.beginPath();
			theContext.moveTo(0, baselineY - psi.im[0]*pxPerY);
			for (var i=1; i<=iMax; i++) {
				theContext.lineTo(i, baselineY - psi.im[i]*pxPerY);
			}
			theContext.strokeStyle = "#00d0ff";
			theContext.stroke();

		} else {	// "Density/phase" is checked

			// Plot the probability distribution with phase as color:
			baselineY = theCanvas.height * (1 - clockSpaceFraction);
			pxPerY = baselineY * 0.55;
			theContext.lineWidth = 2;
			for (var i=0; i<=iMax; i++) {
				theContext.beginPath();
				theContext.moveTo(i, baselineY);
				theContext.lineTo(i, baselineY - pxPerY*(psi.re[i]*psi.re[i] + psi.im[i]*psi.im[i]));
				var localPhase = Math.atan2(psi.im[i], psi.re[i]);
				if (localPhase < 0) localPhase += 2*Math.PI;
				theContext.strokeStyle = phaseColor[Math.round(localPhase * nColors / (2*Math.PI))];
				theContext.stroke();
			}
		}

		// Draw the eigen-phasor diagrams:
		var phasorSpace = theCanvas.height * clockSpaceFraction;
		var clockRadius = phasorSpace * clockRadiusFraction;
		for (var n=0; n<=nMax; n++) {
			theContext.strokeStyle = "gray";
			theContext.lineWidth = 1;
			theContext.beginPath();
			var centerX = (n+0.5)*phasorSpace;
			var centerY = theCanvas.height - 0.5*phasorSpace;
			theContext.arc(centerX, centerY, clockRadius, 0, 2*Math.PI);
			theContext.stroke();
			theContext.beginPath();
			theContext.moveTo(centerX, centerY);
			var clockHandX = centerX + clockRadius*amplitude[n]*Math.cos(phase[n]);
			var clockHandY = centerY - clockRadius*amplitude[n]*Math.sin(phase[n]);
			theContext.lineTo(clockHandX, clockHandY);
			theContext.strokeStyle = phaseColor[Math.round(phase[n] * nColors / (2*Math.PI))];
			theContext.lineWidth = 3;
			theContext.stroke();
		}

		// Provide feedback when setting an amplitude:
		if (mouseIsDown) {
			theContext.fillStyle = "#a0a0a0";
			theContext.font = "20px monospace";
			theContext.fillText("n = " + mouseClock, 100, 30);
			var amp = amplitude[mouseClock];
			var ph = phase[mouseClock];
			theContext.fillText("Mag = " + Number(amp).toFixed(3), 195, 30);
			var deg = String.fromCharCode(parseInt('00b0',16));		// degree symbol
			theContext.fillText("Phase = " + Math.round(ph*180/Math.PI) + deg, 360, 30);
			//theContext.fillText("Re = " + Number(amp*Math.cos[ph]).toFixed(3), 180, 30);
		}
	}

	function startStop() {
		running = !running;
		if (running) {
			pauseButton.innerHTML = "Pause";
			nextFrame();
		} else {
			pauseButton.innerHTML = "Resume";
		}
	}

	function zero() {
		for (var n=0; n<=nMax; n++) {
			amplitude[n] = 0;
		}
		buildPsi();
		paintCanvas();
	}

	function normalizePsi() {
		var norm2 = 0;
		for (var n=0; n<=nMax; n++) {
			norm2 += amplitude[n] * amplitude[n];
		}
		if (norm2 > 0) {
			for (var n=0; n<=nMax; n++) {
				amplitude[n] /= Math.sqrt(norm2);
			}
			buildPsi();
			paintCanvas();
		}
	}

	function coherent() {
		var alphaMag = Number(alphaSlider.value);
		var nFact = 1;
		for (var n=0; n<=nMax; n++) {
			if (n > 0) nFact *= n;
			amplitude[n] = Math.pow(alphaMag, n) / Math.sqrt(nFact);
			phase[n] = 0;
		}
		normalizePsi();
	}

	function adjustAlpha() {
		alphaReadout.innerHTML = Number(alphaSlider.value).toFixed(1);
	}

	// Function to convert a number to a two-digit hex string (from stackoverflow):
	function twoDigitHex(c) {
		var hex = c.toString(16);
		return hex.length == 1 ? "0" + hex : hex;
	}

	// Function to create a hex color string for a given hue (between 0 and 1):
	function colorString(hue) {
		var r, g, b;
		if (hue < 1/6) {
			r = 255; g = Math.round(hue*6*255); b = 0;			// red to yellow
		} else if (hue < 1/3) {
			r = Math.round((1/3 - hue)*6*255); g = 255; b = 0;	// yellow to green
		} else if (hue < 1/2) {
			r = 0; g = 255; b = Math.round((hue - 1/3)*6*255);	// green to cyan
		} else if (hue < 2/3) {
			r = 0; g = Math.round((2/3 - hue)*6*255); b = 255;	// cyan to blue
		} else if (hue < 5/6) {
			r = Math.round((hue - 2/3)*6*255); g = 0; b = 255;	// blue to magenta
		} else {
			r = 255; g = 0; b = Math.round((1 - hue)*6*255);	// magenta to red
		}
		return "#" + twoDigitHex(r) + twoDigitHex(g) + twoDigitHex(b);
	}

</script>


</body>

</html>
"""

f.write(html_template)


f.close()



IPython.display.HTML(filename='/content/GFG.html')
#code in for graph and other python is embedded in html file